In [23]:
import pandas as pd

In [24]:
df = pd.read_csv('Marathi_LDC_Train.csv')

In [25]:
df.head(10)

,Text,Label
0,पणजी : वाहन अपघातामुळे एखाद्या व्यक्तीचा बळी ग...,Auto
1,नवी दिल्ली : साधारण शंभर वर्षांपूर्वी बाजारात ...,Auto
2,नवी दिल्ली : लक्झरी कार निर्माता लेक्सस (Lexus...,Auto
3,निस्सान या जपानच्या वाहन निर्माता कंपनीने भारत...,Auto
4,सुरत : इच्छा तिथे मार्ग!हे सुरतच्या एका 60 वर्...,Auto
5,"शहरात दररोज काचे भरपूर रनिंग होणार आहे, वाहतूक...",Auto
6,Yamaha Motor India (यामाहा मोटर इंडिया) ने गुर...,Auto
7,कर्मचाऱ्यांना त्यांच्या कष्टाचे चीज म्हणून एखा...,Auto
8,नवी दिल्ली-देशातील प्रसिद्ध उद्योगपती आनंद महि...,Auto
9,मारुती सुझुकी या कंपनीनं सप्टेंबर २०२० मध्ये आ...,Auto


In [26]:
df['Label'].value_counts()

Manoranjan       2986
Crime            2354
Politics         2250
Tech             2111
International    2041
Health           1985
Fashion          1920
Sports           1882
Auto             1664
Bhakti           1386
Travel            755
Education         680
Label              11
Name: Label, dtype: int64

In [27]:
df.isnull().sum()

Text     1589
Label       0
dtype: int64

In [28]:
df.dropna(axis=0,inplace=True)

In [39]:
import pandas as pd

target_sample = {
    'Manoranjan': 1500,
    'Politics': 1300,
    'Auto': 1300,
    'Tech': 1300,
    'International': 1300
}


df_sampled = pd.DataFrame()
for label, count in target_sample.items():
    df_label = df[df['Label'] == label].sample(n=count, random_state=42)
    df_sampled = pd.concat([df_sampled, df_label], axis=0)

df_sampled = df_sampled.sample(frac=1).reset_index(drop=True) 

In [40]:
df_sampled

,Text,Label
0,Maharashtra Politics: महाराष्ट्रात येणारा वेदा...,Politics
1,Shiv Sena Dasara Melava : पाकिस्तानमधील माजी प...,Politics
2,Kabul School Blast:अफगाणिस्तानची (Afghanistan)...,International
3,Benelli India ने भारतात Benelli Imperiale 400 ...,Auto
4,नवी दिल्ली-स्मार्टफोनच्या बाजारपेठेत आता बहुता...,Tech
...,...,...
6695,Nokia दरवर्षी MWC इव्हेंटमध्ये एकाच वेळी अनेक ...,Tech
6696,नवी दिल्ली : नवीन कार लाँच करूनही टाटा मोटर्सल...,Auto
6697,कोलंबो - भारताने व्यक्त केलेल्या चिंतेकडे दुर्...,International
6698,Gujarat Assembly Election 2022: गुजरात विधानसभ...,Manoranjan


In [41]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

vectorizer = CountVectorizer(max_df=0.95, min_df=2)
X = vectorizer.fit_transform(df_sampled['Text'].astype(str))

n_topics = 5
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(X)

feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    top_words = " ".join([feature_names[i] for i in topic.argsort()[:-11:-1]])
    print(f"Topic {topic_idx+1}: {top_words}")


doc_topic_distribution = lda.transform(X)

threshold = 0.40
multi_labels = []

for probs in doc_topic_distribution:
    topics = []
    for idx, prob in enumerate(probs):
        if prob >= threshold:
            topics.append(f'Topic_{idx+1}')
    if not topics:
        topics.append(f'Topic_{np.argmax(probs)+1}')
    multi_labels.append(topics)

df_sampled['Multi_Labels'] = multi_labels

print(df_sampled[['Text', 'Multi_Labels']].head())

Topic 1: आण आल पन टर पर पय एक मध सर वर
Topic 2: वर कर अस मह आण सरक तर करण रत पर
Topic 3: आण वर रम कर खल रपट अन अभ गल मह
Topic 4: कर वर अस तर वस आण मह यम असत आत
Topic 5: वर तर आण रश आल एक मध करण रत सर
                                                Text Multi_Labels
0  Maharashtra Politics: महाराष्ट्रात येणारा वेदा...    [Topic_4]
1  Shiv Sena Dasara Melava : पाकिस्तानमधील माजी प...    [Topic_4]
2  Kabul School Blast:अफगाणिस्तानची (Afghanistan)...    [Topic_5]
3  Benelli India ने भारतात Benelli Imperiale 400 ...    [Topic_1]
4  नवी दिल्ली-स्मार्टफोनच्या बाजारपेठेत आता बहुता...    [Topic_1]


In [42]:
df_sampled

,Text,Label,Multi_Labels
0,Maharashtra Politics: महाराष्ट्रात येणारा वेदा...,Politics,[Topic_4]
1,Shiv Sena Dasara Melava : पाकिस्तानमधील माजी प...,Politics,[Topic_4]
2,Kabul School Blast:अफगाणिस्तानची (Afghanistan)...,International,[Topic_5]
3,Benelli India ने भारतात Benelli Imperiale 400 ...,Auto,[Topic_1]
4,नवी दिल्ली-स्मार्टफोनच्या बाजारपेठेत आता बहुता...,Tech,[Topic_1]
...,...,...,...
6695,Nokia दरवर्षी MWC इव्हेंटमध्ये एकाच वेळी अनेक ...,Tech,[Topic_1]
6696,नवी दिल्ली : नवीन कार लाँच करूनही टाटा मोटर्सल...,Auto,[Topic_1]
6697,कोलंबो - भारताने व्यक्त केलेल्या चिंतेकडे दुर्...,International,[Topic_5]
6698,Gujarat Assembly Election 2022: गुजरात विधानसभ...,Manoranjan,[Topic_5]


In [46]:
import pandas as pd
import numpy as np

# 🔹 Synthetic Test Data
synthetic_data = [
    {'Text': 'बॉलीवूड अभिनेत्याचा नवीन चित्रपट प्रदर्शित, प्रेक्षकांची गर्दी'},          # Manoranjan
    {'Text': 'लोकसभा निवडणुकीत मोठा बदल, नवीन उमेदवारांचा दबदबा'},                      # Politics
    {'Text': 'टेस्ला इलेक्ट्रिक कार भारतात लवकरच लॉन्च होणार'},                             # Auto, Tech
    {'Text': 'जगातील सर्वात वेगवान सुपरकॉम्प्युटर विकसित, भारताचा सहभाग'},                 # Tech, International
    {'Text': 'अमेरिकेच्या राष्ट्राध्यक्षांनी नवे धोरण जाहीर केले'},                         # International, Politics
    {'Text': 'महिंद्राने नवी SUV बाजारात आणली, दमदार वैशिष्ट्ये'},                         # Auto
    {'Text': 'फिल्मफेअर पुरस्कारात मराठी चित्रपटांची चमक'},     # Manoranjan
    {'Text':'मुख्यमंत्र्यांनी नवीन योजना जाहीर केली'},
    {'Text':'बॉलिवूड चित्रपटाच्या प्रदर्शनाला राजकारण्यांची उपस्थिती'},
    {'Text':'अमेरिकेच्या राष्ट्राध्यक्षांचा भारत दौरा, तंत्रज्ञान करारावर स्वाक्षरी'}
]

# 🔹 Topic to Label mapping
topic_label_mapping = {
    'Topic_1': 'Auto',
    'Topic_2': 'Tech',
    'Topic_3': 'Manoranjan',
    'Topic_4': 'Politics',
    'Topic_5': 'International'
}

test_df = pd.DataFrame(synthetic_data)

# 🔹 Vectorize test data using the same vectorizer
X_test = vectorizer.transform(test_df['Text'].astype(str))

# 🔹 Get topic probabilities for test data
test_doc_topic_distribution = lda.transform(X_test)

# 🔹 Multi-label assignment based on threshold
threshold = 0.20  
test_multi_labels = []

for probs in test_doc_topic_distribution:
    topics = []
    for idx, prob in enumerate(probs):
        if prob >= threshold:
            topics.append(f'Topic_{idx+1}')
    if not topics:
        topics.append(f'Topic_{np.argmax(probs)+1}')
    test_multi_labels.append(topics)

# 🔹 Map LDA Topics to Original Labels
test_df['Multi_Labels'] = test_multi_labels
test_df['Predicted_Labels'] = test_df['Multi_Labels'].apply(
    lambda topics: [topic_label_mapping.get(topic, 'Unknown') for topic in topics]
)

print(test_df[['Text', 'Predicted_Labels']])

                                                Text        Predicted_Labels
0  बॉलीवूड अभिनेत्याचा नवीन चित्रपट प्रदर्शित, प्...            [Manoranjan]
1  लोकसभा निवडणुकीत मोठा बदल, नवीन उमेदवारांचा दबदबा   [Auto, International]
2     टेस्ला इलेक्ट्रिक कार भारतात लवकरच लॉन्च होणार                  [Auto]
3  जगातील सर्वात वेगवान सुपरकॉम्प्युटर विकसित, भा...            [Auto, Tech]
4  अमेरिकेच्या राष्ट्राध्यक्षांनी नवे धोरण जाहीर ...         [International]
5  महिंद्राने नवी SUV बाजारात आणली, दमदार वैशिष्ट्ये                  [Auto]
6         फिल्मफेअर पुरस्कारात मराठी चित्रपटांची चमक  [Manoranjan, Politics]
7             मुख्यमंत्र्यांनी नवीन योजना जाहीर केली                  [Tech]
8  बॉलिवूड चित्रपटाच्या प्रदर्शनाला राजकारण्यांची...  [Manoranjan, Politics]
9  अमेरिकेच्या राष्ट्राध्यक्षांचा भारत दौरा, तंत्...   [Tech, International]
